In [1]:
# Gym stuff
import gym
import gym_anytrading
from gym_anytrading.envs import StocksEnv, ForexEnv


# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO

# Technicals
from finta import TA

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# Project Modules
from collections import deque
from config import LOG_PATH, MODEL_PATH
import cryptomanager as cm
import assetpicker as ap
from datetime import datetime
from IPython.display import clear_output
import os
import time
from trader import trader_agent

# The class that will do the work of trading
trade_me = trader_agent()

In [2]:
# Initial DataFrame to train on
df = ap.historical_data_df(days_delta=10000)
df



start date:  1995-07-07 00:00:00


,Date,Open,High,Low,Close,Volume,Trade_Count,VWAP
0,2015-12-01 09:04,209.4300,209.4300,209.430,209.4300,470.0,2.0,209.430000
1,2015-12-01 09:05,209.3500,209.3500,209.320,209.3200,1160.0,3.0,209.322586
2,2015-12-01 09:06,209.4200,209.4200,209.420,209.4200,100.0,1.0,209.420000
3,2015-12-01 09:18,209.4400,209.4400,209.440,209.4400,1000.0,1.0,209.440000
4,2015-12-01 09:19,209.4300,209.4300,209.420,209.4300,800.0,4.0,209.427500
...,...,...,...,...,...,...,...,...
1394499,2022-11-22 16:20,397.4189,397.4300,397.250,397.4257,90292.0,885.0,397.324138
1394500,2022-11-22 16:21,397.4200,397.4900,397.375,397.4000,62863.0,790.0,397.430801
1394501,2022-11-22 16:22,397.4100,397.4900,397.290,397.3538,63380.0,811.0,397.402686
1394502,2022-11-22 16:23,397.3500,397.3599,397.250,397.3400,58788.0,616.0,397.306116


In [3]:
# Initial DataFrame to train on
df = ap.historical_data_df(days_delta=10000)
df

start date:  1995-07-07 00:00:00


,Date,Open,High,Low,Close,Volume,Trade_Count,VWAP
0,2015-12-01 09:04,209.430,209.430,209.4300,209.4300,470.0,2.0,209.430000
1,2015-12-01 09:05,209.350,209.350,209.3200,209.3200,1160.0,3.0,209.322586
2,2015-12-01 09:06,209.420,209.420,209.4200,209.4200,100.0,1.0,209.420000
3,2015-12-01 09:18,209.440,209.440,209.4400,209.4400,1000.0,1.0,209.440000
4,2015-12-01 09:19,209.430,209.430,209.4200,209.4300,800.0,4.0,209.427500
...,...,...,...,...,...,...,...,...
1394507,2022-11-22 16:28,397.325,397.410,397.2101,397.3800,109263.0,1212.0,397.311544
1394508,2022-11-22 16:29,397.400,397.480,397.3000,397.3600,51337.0,767.0,397.383410
1394509,2022-11-22 16:30,397.350,397.385,397.2600,397.3599,54873.0,683.0,397.306994
1394510,2022-11-22 16:31,397.340,397.370,397.2800,397.3100,36626.0,527.0,397.322402


In [4]:
# The range that we want to train on each interval
window_size = 1 
lower_bound = window_size
upper_bound = len(df)
df[lower_bound:upper_bound]

,Date,Open,High,Low,Close,Volume,Trade_Count,VWAP
1,2015-12-01 09:05,209.350,209.350,209.3200,209.3200,1160.0,3.0,209.322586
2,2015-12-01 09:06,209.420,209.420,209.4200,209.4200,100.0,1.0,209.420000
3,2015-12-01 09:18,209.440,209.440,209.4400,209.4400,1000.0,1.0,209.440000
4,2015-12-01 09:19,209.430,209.430,209.4200,209.4300,800.0,4.0,209.427500
5,2015-12-01 09:20,209.430,209.430,209.4200,209.4200,520.0,6.0,209.424615
...,...,...,...,...,...,...,...,...
1394507,2022-11-22 16:28,397.325,397.410,397.2101,397.3800,109263.0,1212.0,397.311544
1394508,2022-11-22 16:29,397.400,397.480,397.3000,397.3600,51337.0,767.0,397.383410
1394509,2022-11-22 16:30,397.350,397.385,397.2600,397.3599,54873.0,683.0,397.306994
1394510,2022-11-22 16:31,397.340,397.370,397.2800,397.3100,36626.0,527.0,397.322402


In [5]:
# Adding technicals to DataFrame

def add_columns_df(df):
    df['OBV'] = TA.OBV(df) # unsure if I want to use this yet or not
    df['EMA'] = TA.EMA(df, 200)
    df['RSI'] = TA.RSI(df)
    df  = df.join(TA.PIVOT_FIB(df))
    df = df.join(TA.MACD(df))
    df.fillna(0, inplace=True)
    return df
    
df = add_columns_df(df)
df.head()

,Date,Open,High,Low,Close,Volume,Trade_Count,VWAP,OBV,EMA,...,s1,s2,s3,s4,r1,r2,r3,r4,MACD,SIGNAL
0,2015-12-01 09:04,209.43,209.43,209.43,209.43,470.0,2.0,209.430000,0.0,209.430000,...,0.00000,0.00000,0.00,0.00000,0.00000,0.00000,0.00,0.00000,0.000000,0.000000
1,2015-12-01 09:05,209.35,209.35,209.32,209.32,1160.0,3.0,209.322586,-1160.0,209.374725,...,209.43000,209.43000,209.43,209.43000,209.43000,209.43000,209.43,209.43000,-0.002468,-0.001371
2,2015-12-01 09:06,209.42,209.42,209.42,209.42,100.0,1.0,209.420000,-1060.0,209.389968,...,209.31854,209.31146,209.30,209.28854,209.34146,209.34854,209.36,209.37146,-0.000044,-0.000827
3,2015-12-01 09:18,209.44,209.44,209.44,209.44,1000.0,1.0,209.440000,-60.0,209.402664,...,209.42000,209.42000,209.42,209.42000,209.42000,209.42000,209.42,209.42000,0.001782,0.000057
4,2015-12-01 09:19,209.43,209.43,209.42,209.43,800.0,4.0,209.427500,-860.0,209.408241,...,209.44000,209.44000,209.44,209.44000,209.44000,209.44000,209.44,209.44000,0.002339,0.000736


In [6]:
# Adding metrics to Env

def add_signals(env, df = df):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    keys= []
    for key in df.columns:
        keys.append(key)
    signal_features = env.df.loc[:, [key for key in keys[1:]]].to_numpy()[start:end]
    return prices, signal_features

In [7]:
class Cypto_Env(StocksEnv):
    _process_data = add_signals
    def __init__(self, df, window_size, frame_bound):
        super().__init__(df, window_size, frame_bound)
        self.trade_fee_bid_percent = 0  # unit
        self.trade_fee_ask_percent = 0  # unit
    

In [8]:
# The initial training Env before we swtich to up-to-date Env

env2 = Cypto_Env(df=df, window_size=window_size, frame_bound=(lower_bound,upper_bound))
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [9]:
 
# create and train prediction model

episodes = 5
step_count = 1000

# model = A2C.load(path="./data/model/1667783635.2006977", env = env)
model = A2C('MlpPolicy', env, verbose=1, device="cuda") 

name = str(datetime.now().strftime("%Y-%d-%m-%H-%M") )

for i in range (episodes):
    model = model.learn(total_timesteps=step_count, 
    ) 
    print(i)

Using cpu device
-------------------------------------
| time/                 |           |
|    fps                | 591       |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -0.652    |
|    explained_variance | -1.44     |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -0.000459 |
|    value_loss         | 1.99e-05  |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 615      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -0.69    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | 0.0167   |
|    va

In [10]:
prev_min = 0
print(len(df)-lower_bound,len(df))

# stop double selling
has_sold = True

1394511 1394512


In [11]:

df_temp = cm.historical_data_df()


df_temp = add_columns_df(df_temp)

df_temp = pd.concat([df, df_temp], ignore_index=True, sort=False)

df_temp.iloc[:,0]
past_trades = deque(maxlen=60)


In [12]:
cur_min = datetime.now().minute
prev_min = cur_min
test_env = Cypto_Env(df=df, window_size=window_size, frame_bound=(lower_bound,upper_bound))
wait_time = window_size
min_elapsed = 0
def rebalance_df():
    df_temp = cm.historical_data_df()

    df_temp = add_columns_df(df_temp)
    df_temp = pd.concat([df, df_temp], ignore_index=True, sort=False)

    test_env = Cypto_Env(df=df_temp, window_size=window_size, frame_bound=(window_size,upper_bound))
    return df_temp

def return_obs(df):
    start = len(df) - window_size
    end = len(df)
    keys= []
    for key in df.columns:
        keys.append(key)
    signal_features = df.loc[:, [key for key in keys[1:]]].to_numpy()[start:end]
    return signal_features

while True:
    try:
        cur_min = datetime.now().minute
        if cur_min == 0:
            prev_min = 0
        if cur_min % 5 == 0 and cur_min > prev_min:
            clear_output()

            min_elapsed += wait_time

            print("mins til model update: ", window_size - min_elapsed)
            cur_min = datetime.now().minute
            prev_min = cur_min + wait_time - 1

            df_temp = rebalance_df()
            
            obs = return_obs(df_temp)

            action, _ = model.predict(obs)
            print("made a prediction")
            print(action)
            print("Last was sold: ",has_sold)
            if action == 1:
                has_sold = False
                # if market is closed, buy BTCUSD
                # check market time
                isOpen = trade_me.isOpen

                if not isOpen:
                    # trade_me.buy_position_at_market(ticker = "BTC/USD")
                    print("market is closed")
                else:
                    trade_me.buy_position_at_market(ticker = "SPY")
                print("b") 
            elif ( action == 0 ):
                has_sold = True
                # if market is closed, sell BTCUSD
                if not isOpen:
                    # trade_me.sell_position_at_market(ticker = "BTCUSD")
                    print("market is closed")
                else:
                    trade_me.sell_position_limit(ticker = "SPY")
                print("s")
            past_trades.append(action)
            print("past trades: ", list(past_trades))
            #   print(df_temp.iloc[-window_size:])
            if min_elapsed >= window_size:
                min_elapsed = 0
            for episode in range(episodes):
                model.learn(total_timesteps=step_count, 
                )
                        # model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
                print(episode +1)

    except Exception as e:
        print(e)   
        print("past trades: ", list(past_trades))
        # min_elapsed += wait_time
        if min_elapsed >= window_size:
            min_elapsed = 0
        for episode in range(episodes):
            model.learn(total_timesteps=step_count, 
            )

                # model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
            print(episode +1)



mins til model update:  0
made a prediction
1
Last was sold:  False
market is closed
b
past trades:  [array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), array(1, dtype=int64), a

In [ ]:
# Save the Model
model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
past_trades

In [ ]:
def Crypto_USD() -> list:

    
    output = []
    cryptos = cm.get_cryptos_tradable()
    for crypto in cryptos:
        if crypto.split("/")[1] == "USD":
            output.append(crypto)
    return output

USD_crypto =  Crypto_USD()   
# USD_crypto.pop(Crypto_USD().index("BTC/USD"))
print(len(USD_crypto))


In [ ]:
# get data frame for each crypto
df_list = []
for crypto in USD_crypto:
    try:
        df_temp = cm.historical_data_df(days_delta=0, crypto=[crypto])
        df_temp = add_columns_df(df)
        df_list.append(df_temp)
    except:
        continue



In [ ]:
# create the environments
init_envs = []
for df_single in df_list:
    env2 = Cypto_Env(df=df_single, window_size=window_size, frame_bound=(lower_bound,upper_bound))
    env_maker = lambda: env2
    env = DummyVecEnv([env_maker])
    init_envs.append(env2)
    

In [ ]:
print(init_envs)

In [ ]:
# train model for each env
models = []
for env_single in init_envs: 
    model_temp = A2C('MlpPolicy', env, verbose=0, device="cuda") 

    name = str(datetime.now().strftime("%Y-%d-%m-%H-%M") )

    episodes = 1
    step_count = 1000
    for i in range (episodes):
        model_temp = model_temp.learn(total_timesteps=step_count, 
        ) 
        print(i)
    models.append(model_temp)
    
    

In [ ]:
print(models)

In [ ]:
prev_min = 0
print(len(df)-lower_bound,len(df))

# stop double selling
has_sold = False

In [ ]:
past_trades = deque(maxlen=60)
cur_min = datetime.now().minute
prev_min = 0
# test_env = Cypto_Env(df=df, window_size=window_size, frame_bound=(lower_bound,upper_bound))
wait_time = 1
while True:
    try:
        cur_min = datetime.now().minute
        if cur_min > prev_min or cur_min == 1:
            cur_min = datetime.now().minute
            prev_min = cur_min + wait_time - 1
            for i, model in enumerate(models):
                print("made it this far, looking for df")
                clear_output()
                cur_crypto = USD_crypto[i]
                print(cur_crypto)
                df_temp = cm.historical_data_df(crypto=[cur_crypto])

                df_temp = add_columns_df(df_temp)

                df_temp = pd.concat([df_list[i], df_temp], ignore_index=True, sort=False)
                
                test_env = Cypto_Env(df=df_temp, window_size=window_size, frame_bound=(window_size,upper_bound))
                
                def return_obs(df):
                    start = len(df) - window_size
                    end = len(df)
                    keys= []
                    for key in df.columns:
                        keys.append(key)
                    signal_features = df.loc[:, [key for key in keys[1:]]].to_numpy()[start:end]
                    return signal_features
                
                # clear_output()
                obs = return_obs(df)
                action = model.predict(obs)
                print("made a prediction on ", cur_crypto)
                print(action)
                if action == 1 or action[0] == 1:
                    trade_me.buy_position_at_market(cur_crypto)
                    print("b") 
                    has_sold = False
                if not has_sold and( action == 0 or action[0] == 0):
                    trade_me.sell_position_market(cur_crypto)
                    print("s")
                    has_sold = True
                # past_trades.append([action, ("price: ", df["Low"].iloc[-1]), ("time: ", datetime.now())])
                # for episode in range(episodes):
                #     model = model.learn(total_timesteps=step_count, 
                #     )
                    # model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
                    # print("past trades: ", list(past_trades))
    except Exception as e:
        print(e)
        # for episode in range(episodes):
        #     model = model.learn(total_timesteps=step_count,
        #      )
            # model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
            # print("past trades: ", list(past_trades))

